In [18]:
from libraries import *
from parameters import *
import warnings
warnings.filterwarnings('ignore')
#from statannot import add_stat_annotation

%load_ext rpy2.ipython

import rpy2.robjects as ro
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri


The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [2]:
setIndex_1=0

In [3]:
os.getcwd()
os.chdir(projectDir)

In [4]:
setIndex_2= setIndex_1+10

In [5]:
adata = sc.read('outputs/anndata/adata-hash-features_singlets_SingleKO_08122020_PerGENE.h5ad')

In [6]:
covariates = adata.uns['feature_barcode_names_filtered_GENES']
covariates = covariates[covariates != "GENE_CONTROL_"]

guideMatrix = adata.obs[covariates]
KOGenes = pd.Series(guideMatrix.columns)

## Add covariates that should be corrected for
df = adata.obs[["n_genes", "mt_frac", "leiden" ]]

guideMatrix = guideMatrix.join(df)
guideMatrix.shape

(242938, 1034)

In [7]:
expressionMatrix = pd.DataFrame(adata.raw.X.A)
expressionMatrix.columns = adata.uns['AllExpGeneNames'].iloc[:,0]
expressionMatrix = expressionMatrix[adata.var_names]
expressionMatrix.index = adata.obs.index

In [8]:
selColumns = pd.Series(KOGenes[setIndex_1:setIndex_2]).append(pd.Series(["n_genes", "mt_frac", "leiden"]))

In [9]:
controlExpressionMat = expressionMatrix.loc[adata.obs["GENE_CONTROL_"] == 1,]
controlGuideMat = guideMatrix.loc[adata.obs["GENE_CONTROL_"] == 1,]
controlExpressionMat = controlExpressionMat.iloc[0:5000,:]
controlGuideMat = controlGuideMat.iloc[0:5000,:]
controlGuideMat=controlGuideMat[selColumns]

geneExpressionMat = expressionMatrix.loc[adata.obs[covariates[setIndex_1:setIndex_2]].sum(axis=1) > 0,]
geneGuideMat = guideMatrix.loc[adata.obs[covariates[setIndex_1:setIndex_2]].sum(axis=1) > 0,]
geneGuideMat=geneGuideMat[selColumns]

In [10]:
allExpMat = np.concatenate((controlExpressionMat, geneExpressionMat))
allGuideMat = controlGuideMat.append(geneGuideMat)

In [11]:
allVars = pd.Series(KOGenes[setIndex_1:setIndex_2]) 

all_columns = "+".join(allVars[~allVars.isin(["leiden"])])
all_columns = all_columns +  "+n_genes+mt_frac"

my_formula = "y~" + all_columns + " + (1 | leiden)"
my_formula

'y~GENE_4921501E09Rik_+GENE_4930595M18Rik_+GENE_8030462N17Rik_+GENE_Aamp_+GENE_Abtb1_+GENE_Abtb2_+GENE_Acaca_+GENE_Ago2_+GENE_Ahctf1_+GENE_Ahr_+n_genes+mt_frac + (1 | leiden)'

In [12]:
allResp=adata.var_names

In [13]:
if not os.path.exists("./TmpMixedEffectNBLMOuts/"):
    os.mkdir("./TmpMixedEffectNBLMOuts/")

In [29]:
allGuideMat

,GENE_4921501E09Rik_,GENE_4930595M18Rik_,GENE_8030462N17Rik_,GENE_Aamp_,GENE_Abtb1_,GENE_Abtb2_,GENE_Acaca_,GENE_Ago2_,GENE_Ahctf1_,GENE_Ahr_,n_genes,mt_frac,leiden
AAACGCTAGTTGTAGA-1-E3-1,0,0,0,0,0,0,0,0,0,0,951,0.086056,0
AAAGGATTCAGACAAA-1-E3-1,0,0,0,0,0,0,0,0,0,0,1247,0.071198,2
AAAGGGCAGACGGTTG-1-E3-1,0,0,0,0,0,0,0,0,0,0,1514,0.073052,1
AAAGGGCAGAGATCGC-1-E3-1,0,0,0,0,0,0,0,0,0,0,1561,0.062370,0
AAAGGGCGTTGATGTC-1-E3-1,0,0,0,0,0,0,0,0,0,0,1095,0.051056,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
TGCGATAGTGTTCGTA-1-E4-24,0,0,0,0,0,1,0,0,0,0,1769,0.056831,0
TTAGGGTGTGAGTTTC-1-E4-24,0,0,0,0,0,0,0,0,1,0,2340,0.076101,2
TTCATTGTCTGAACGT-1-E4-24,1,0,0,0,0,0,0,0,0,0,1945,0.059586,2
TTCTGTAAGTGTAGTA-1-E4-24,0,0,0,0,0,0,0,1,0,0,1636,0.099162,1


In [38]:
allExpMat = pd.DataFrame(allExpMat)

In [40]:
allExpMat.columns = allResp

In [41]:
allExpMat

,Mrpl15,Lypla1,Tcea1,Atp6v1h,Rb1cc1,Pcmtd1,Rrs1,Vcpip1,Sgk3,Snhg6,...,Cacul1,Eif3a,Fam45a,Prdx3,Grk5,Zfp950,Csf2ra,Vamp7,AC149090.1,CAAA01118383.1
0,1.678814,0.000000,2.274021,1.678814,0.000000,0.000000,1.678814,0.000000,0.0,0.0,...,0.000000,1.678814,0.000000,0.0,0.0,0.0,1.678814,0.0,0.000000,0.00000
1,0.000000,0.000000,1.424210,1.424210,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,1.989127,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.00000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.253478,0.000000,0.0,0.0,...,0.000000,2.398805,0.000000,0.0,0.0,0.0,1.253478,0.0,1.253478,0.00000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.166406,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,1.690256,0.0,0.000000,0.00000
4,0.000000,0.000000,0.000000,1.463778,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,2.033989,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7218,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.994040,0.000000,0.0,0.0,0.0,0.994040,0.0,0.000000,0.00000
7219,0.000000,0.000000,0.742509,0.742509,0.742509,0.000000,0.000000,0.000000,0.0,0.0,...,0.742509,2.029095,0.742509,0.0,0.0,0.0,1.163901,0.0,0.000000,0.00000
7220,0.902884,0.000000,0.000000,0.000000,0.902884,0.000000,0.000000,0.902884,0.0,0.0,...,0.000000,1.686421,0.000000,0.0,0.0,0.0,1.686421,0.0,0.000000,0.00000
7221,0.000000,0.000000,0.000000,0.000000,0.000000,1.091990,1.091990,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,1.091990,0.0,1.091990,1.09199


In [42]:
r_allGuideMat = ro.conversion.get_conversion().py2rpy(allGuideMat)
r_allExpMat = ro.conversion.get_conversion().py2rpy(allExpMat)
r_my_formula = ro.conversion.get_conversion().py2rpy(my_formula)
#r_allResp = ro.conversion.get_conversion().py2rpy(allResp)
r_setIndex_1 = ro.conversion.get_conversion().py2rpy(setIndex_1)

In [34]:
allResp

Index(['Mrpl15', 'Lypla1', 'Tcea1', 'Atp6v1h', 'Rb1cc1', 'Pcmtd1', 'Rrs1',
       'Vcpip1', 'Sgk3', 'Snhg6',
       ...
       'Cacul1', 'Eif3a', 'Fam45a', 'Prdx3', 'Grk5', 'Zfp950', 'Csf2ra',
       'Vamp7', 'AC149090.1', 'CAAA01118383.1'],
      dtype='object', length=6685)

In [43]:
myDatframe = {}
myDatframe['allGuideMat'] = r_allGuideMat
myDatframe['allExpMat'] = r_allExpMat
myDatframe['my_formula'] = r_my_formula
#myDatframe['allResp'] = r_allResp
myDatframe['setIndex_1'] = r_setIndex_1
MyRDataframe = ro.DataFrame(myDatframe)
MyRDataframe

allGuideMat.GENE_4921501E09Rik_,allGuideMat.GENE_4930595M18Rik_,allGuideMat.GENE_8030462N17Rik_,...,allExpMat.CAAA01118383.1,my_formula,setIndex_1
...,...,...,...,...,'y~GENE_4...,...


In [51]:
activity = ro.r(r'''
    function(allGuideMat, allExpMat,my_formula,setIndex_1) {
        library("MASS")
        library("lme4")

        coefDF = data.frame()

        for(i in seq(1,2,1)){


            tryCatch(
                expr = {
                       allGuideMat["y"] = allExpMat[,i]
                       myFit <- glmer.nb(formula(my_formula), 
                                         data = allGuideMat, 
                                         control = glmerControl(optimizer="bobyqa", 
                                                                calc.derivs = FALSE))
                       k <- summary(myFit)
                       myDF <- data.frame(coefficients(k))
                       myDF$coefNames <- dimnames(coefficients(k))[[1]]
                       myDF$respGene = colnames(allExpMat)[i]
                       coefDF <- rbind(coefDF, myDF)
                },
                error = function(e){ 
                    print(e)
                },
                finally = {
                    # (Optional)
                    # Do this at the end before quitting the tryCatch structure...
                }
            )
        }
        
        saveRDS(coefDF, paste0("./TmpMixedEffectNBLMOuts/coefs_",setIndex_1,".rds"))

        }
''')

In [52]:
activity(myDatframe['allGuideMat'], 
         myDatframe['allExpMat'], 
         myDatframe['my_formula'],
         myDatframe['setIndex_1'])

<rpy2.rinterface_lib.sexp.NULLType object at 0x7fc06bfed340> [RTYPES.NILSXP]

In [ ]:
%%R -i allGuideMat,allExpMat,my_formula,allResp,setIndex_1,setIndex_2  -w 8 -h 4 -u in
library("MASS")
library("lme4")

coefDF = data.frame()

for(i in seq(1,2,1)){
    
    tryCatch(
        expr = {
               allGuideMat["y"] = allExpMat[,i]
               myFit <- glmer.nb(formula(my_formula), 
                                 data = allGuideMat,
                                 control = glmerControl(optimizer="bobyqa", 
                                                        calc.derivs = FALSE))
               k <- summary(myFit)
               myDF <- data.frame(coefficients(k))
               myDF$coefNames <- dimnames(coefficients(k))[[1]]
               myDF$respGene = allResp[j]

               coefDF <- rbind(coefDF, myDF)
        },
        error = function(e){ 
            print(e)
        },
        finally = {
            # (Optional)
            # Do this at the end before quitting the tryCatch structure...
        }
    )

    
}

saveRDS(coefDF, paste0("./TmpMixedEffectNBLMOuts/coefs_",setIndex_1,"_",setIndex_2,".rds"))